# Hồi quy tuyến tính đơn giản (Simple Linear Regression)

Trong notebook này, chúng ta sẽ sử dụng dữ liệu doanh số bán nhà ở Quận King để dự đoán giá nhà sử dụng hồi quy tuyến tính đơn giản (1 đầu vào). Chúng ta sẽ:
* Sử dụng thư viện pandas để load tập dữ liệu hiện có.
* Viết một hàm tính toán trọng số của Hồi quy tuyến tính đơn giản với biểu thức dạng đóng.
* Viết một hàm dự đoán đầu ra với đặc trưng đầu vào đã cho.
* Thực hiện hồi quy để dự đoán đầu vào với đầu ra đã cho.
* So sánh 2 mô hình khác nhau dự đoán giá nhà.

Trong notebook này, các bạn sẽ được cung cấp một số code hoàn chỉnh và cũng cần tự tạo code để trả lời các quiz. Code chúng tôi cung cấp giúp bạn hoàn thiện các bài toán là tùy chọn, các bạn có thể bỏ qua helper code và viết code của riêng mình.

## Import thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/house_data.zip","r")
zip_ref.extractall("/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2")
zip_ref.close()

In [ ]:
import sklearn, pandas
import numpy as np

## Load dữ liệu bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [ ]:
full_data = pandas.read_csv("/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/kc_house_data.csv", index_col=0)

In [ ]:
print(full_data.describe())
print(full_data.head(6))
print(full_data.info())

              price      bedrooms     bathrooms   sqft_living      sqft_lot  \
count  2.161300e+04  21613.000000  21613.000000  21613.000000  2.161300e+04   
mean   5.400881e+05      3.370842      2.114757   2079.899736  1.510697e+04   
std    3.671272e+05      0.930062      0.770163    918.440897  4.142051e+04   
min    7.500000e+04      0.000000      0.000000    290.000000  5.200000e+02   
25%    3.219500e+05      3.000000      1.750000   1427.000000  5.040000e+03   
50%    4.500000e+05      3.000000      2.250000   1910.000000  7.618000e+03   
75%    6.450000e+05      4.000000      2.500000   2550.000000  1.068800e+04   
max    7.700000e+06     33.000000      8.000000  13540.000000  1.651359e+06   

             floors    waterfront          view     condition         grade  \
count  21613.000000  21613.000000  21613.000000  21613.000000  21613.000000   
mean       1.494309      0.007542      0.234303      3.409430      7.656873   
std        0.539989      0.086517      0.766318    

## Đọc dữ liệu

Chúng ta sẽ chia dữ liệu thành train/test set. Để duy trì thời gian chạy nhất quán, chúng ta cần một gái trị seed (default = 0).

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

## Các hàm Pandas hữu ích

In order to make use of the closed form solution as well as taking advantage of pandas's built-in capacity, we will do some simple exercise. In particular: Để tận dụng biểu thức dạng đóng cũng như các tiện ích được tích hợp sẵn của pandas, chúng ta cùng làm một số bài tập đơn giản. Cụ thể:
* Tính tổng một cột.
* Tính trung bình cộng của các cột.
* Nhân và cập nhật một cột với một hằng số.
* Nhân một cột với giá trị của một cột khác.

In [ ]:
# Hãy tính mean của House Prices in King County theo 2 cách khác nhau:
prices = full_data['price']

# gọi lại trung bình cộng, là tổng các giá chia tổng số lượng ngôi nhà:
avg_price_auto = prices.mean()
avg_price_manual = sum(prices.values) / float(len(prices))
print("average price (function): %.2f" % (avg_price_auto))
print("average price (manual):  %.2f" % (avg_price_manual))

average price (function): 540088.14
average price (manual):  540088.14


In [ ]:
# nhân từng giá với 0.5 như sau:
half_prices = 0.5*prices
# Hãy tính tổng bình phương giá cả, nhân 2 SArrays có cùng độ dài với *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared là SArray của bình phương và chúng ta muốn thêm vào.
print("the sum of price squared is: %.E" % (sum_prices_squared))

the sum of price squared is: 9E+15


Ngoài ra: Kí hiệu python x.xxe+yy nghĩa là x.xx \* 10^(yy). Ví dụ: 100 = 10^2 = 1*10^2 = 1e2

Thay thế kí hiệu số mũ (%.2E) bằng kí hiệu float (%.4f) và hiển thị sự khác biệt.

## Xây hàm hồi quy tuyến tính đơn giản chung

Với các hàm này, chúng ta có thể sử dụng biểu thức dạng đóng từ bài giảng để tính slope (độ dốc) và intercept (hệ số chặn) cho một hồi quy tuyến tính đơn giản trên các quan sát được lưu trữ dưới dạng cột DataFrame: input_feature, output.

Hoàn thành hàm sau (hoặc viết hàm của riêng mình) để tính slope và intercept của hồi quy tuyến tính đơn giản:

In [ ]:
mm = np.array([5,
               6,
               7])
nn = np.array([7
               ,8
               ,9])

mm = mm.T
nn=nn.T
print(mm,nn)
c = mm@nn.T
d = np.multiply(mm,nn)
print(c)
print(d)

[5 6 7] [7 8 9]
146
[35 48 63]


In [ ]:
def simple_linear_regression(input_feature, output):
    # tính tổng của input_feature và output
    in_feature = input_feature
    out_feature = output
    sum_in = in_feature.sum()
    sum_out = output.sum()
    mean_sum_in = sum_in/float(len(in_feature))
    mean_sum_out = sum_out/float(len(in_feature))
    # tính tích của output và input_feature và mean của nó
    sum_y_x = in_feature@out_feature
    mean_sumy_sumx = (in_feature.sum()*out_feature.sum())/(float(len(in_feature)))
    # tính giá trị bình phương của input_feature và mean của nó
    sumxx = in_feature@in_feature
    mean_sumx_sumx = (in_feature.sum()**2)/(float(len(in_feature)))
    # sử dụng công thức tính slope
    slope = (sum_y_x - mean_sumy_sumx)/(sumxx - mean_sumx_sumx)
    # sử dụng công thức tính intercept
    intercept = mean_sum_out - slope*mean_sum_in
    return (intercept, slope)

Chúng ta có thể kiểm tra xem hàm có hoạt động hay không bằng cách truyền cho nó một thứ gì đó cho chúng ta biết câu trả lời. Cụ thể, có thể tạo một đặc trưng rồi đặt đầu ra chính xác trên một dòng: output = 1 + 1\*input_feature, sau đó chúng ta biết cả slope và intercept là 1

In [ ]:
mock_feature = np.array(range(5))
mock_output = 9 + 35*mock_feature
(mock_intercept, mock_slope) =  simple_linear_regression(mock_feature, mock_output)
print("Intercept: %.2f" % (mock_intercept))
print("Slope: %.2f" % (mock_slope))

Intercept: 9.00
Slope: 35.00


Nó đã hoạt động. Giờ hãy xây dựng mô hình hồi quy để dự đoán giá dựa trên sqft_living. Hãy nhớ chúng ta đang huấn luyện train_data!

In [ ]:
sqft_features = train_data['sqft_living'].values
sqft_labels = train_data['price'].values
sqft_intercept, sqft_slope = simple_linear_regression(sqft_features, sqft_labels)

print("Intercept: %.2f" % (sqft_intercept))
print("Slope: %.2f" % (sqft_slope))

Intercept: -48257.06
Slope: 283.97


# Dự đoán giá

Bây giờ chúng ta có các tham số mô hình: intercept & slope, chúng ta đã có thể đưa ra dự đoán. Với numpy, thật dễ để nhân một mảng numpy với một hằng số và thêm một giá trị không đổi. Hoàn thành hàm sau để trả về đầu ra dự đoán với input_feature, slope và intercept:

In [ ]:
def get_regression_predictions(input_feature, intercept, slope):
      # tính các giá trị đã dự đoán:
    predicted_values = intercept + input_feature*slope
    return predicted_values

Hãy đưa ra dự đoán với slope và intercept. Sử dụng (hoặc thay đổi) thông tin sau để tìm ra giá ước tính cho một ngôi nhà rộng 2,000 feet vuông theo mô hình squarefeet mà chúng ta đã ước tính ở trên.

**Quiz: Với Slope và Intercept từ (4), giá dự đoán cho một ngôi nhà có diện tích 2,000 feet vuông là bao nhiêu?**

In [ ]:
my_house_sqft = 2000
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept,sqft_slope)
print("The estimated price for a house with %d squarefeet is $%.4f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2000 squarefeet is $519680.0507


# Tổng bình phương phần dư (RSS)

Chúng ta đã có một mô hình và có thể đưa ra dự đoán, hãy đánh giá mô hình với RSS. RSS là tổng bình phương phần dư và phần dư chỉ là một từ ngữ hoa mỹ để chỉ hiệu giữa đầu ra được dự đoán và đầu ra thực.

Hoàn thành hàm sau (hoặc tự viết hàm) để tính RSS của một mô hình hồi quy tuyến tính đơn giản với input_feature, output, intercept và slope:

In [ ]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # Trước tiên lấy các dự đoán
    input_array = np.array(input_feature)
    output_array = np.array(output)
    output_predict_array = get_regression_predictions(input_array, intercept,slope)
    print(input_array[0:5])
    print(output_predict_array[:5])
    # sau đó tính các phần dư (chúng ta sẽ bình phương nên thứ tự trừ không quan trọng)
    residu = output_array - output_predict_array
    # bình phương phần dư rồi cộng tổng lại
    RSS = residu@residu
    return(RSS)

Hãy kiểm tra hàm `get_residual_sum_of_squares` bằng cách áp dụng nó vào mô hình kiểm thử, trong đó dữ liệu nằm chính xác trên 1 dòng. Do chúng nằm trên 1 dòng nên RSS sẽ bằng 0!

In [ ]:
print("%.2f" % get_residual_sum_of_squares(mock_feature, mock_output, mock_intercept, mock_slope)) # should be 0.0

[0 1 2 3 4]
[  9.  44.  79. 114. 149.]
0.00


Bây giờ hãy sử dụng hàm của bạn để tính toán RSS trong dữ liệu huấn luyện từ mô hình squarefeet đã tính ở trên.

**Quiz: Với hàm này, slope và intercept của mô hình squarefeet thì RSS cho hồi quy tuyến tính đơn giản sử dụng squarefeet để dự đoán giá trên dữ liệu HUẤN LUYỆN là bao nhiêu?**

In [ ]:
# You have made your bed, time to lie in it
train_features = train_data['sqft_living'].values
train_labels = train_data['price'].values
#sqft_intercept, sqft_slope = simple_linear_regression(sqft_features, sqft_labels)
rss_prices_on_sqft = get_residual_sum_of_squares(train_features, train_labels,sqft_intercept, sqft_slope)
print('The RSS of predicting Prices based on Square Feet is : %.2E' % (rss_prices_on_sqft))

[1570 1780 1090 2210 1800]
[495000. 635000. 382500. 382500. 670000.]
[1570 1780 1090 2210 1800]
[397573.57114253 457206.9681451  261268.66370806 579313.44772181
 462886.33928821]
The RSS of predicting Prices based on Square Feet is : 1.21E+15


# Dự đoán squarefeet với price (giá) đã biết

Vậy nếu chúng ta muốn đoán squarefeet với price đã biết thì sao? Cho phương trình $y = a + b*x$, chúng ta có thể giải hàm tìm x. Như vậy nếu có intercept (a), slope (b) và price (y), có thể tìm squarefeet dự đoán(x).

Hoàn thành hàm sau để tính ước lượng hồi quy ngược, tức là dự đoán input_feature với output đã biết.

In [ ]:
def inverse_regression_predictions(output, intercept, slope):
    # giải output = intercept + slope*input_feature để tìm input_feature. Sử dụng phương trình này để tính dự đoán ngược:
    estimated_feature = (output - intercept)/slope
    return estimated_feature

Chúng ta đã có một hàm để tính squaredeet với giá đã biết từ mô hình hồi quy đơn giản, hãy xem cách dự kiến diện tích của một ngôi nhà với giá 800,000 USD.

**Quiz: Cho hàm này, slope và intercept từ (3), ngôi nhà giá 800,000 USD ước tính rộng bao nhiêu bao nhiêu feet vuông?**

In [ ]:
# kết quả đã có, hãy thêm tham số vào hàm dưới đây
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope) # ở đây
print("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $800000.00 is 2987


# Mô hình mới: ước tính giá từ số phòng ngủ (bedrooms)

Chúng ta đã tạo một mô hình dự đoán giá nhà bằng sử dụng squarefeet, nhưng còn nhiều đặc trưng khác trong dữ liệu bán hàng. Sử dụng hàm hồi quy tuyến tính đơn giản để ước tính các tham số hồi quy từ việc dự đoán Giá nhà dựa vào số lượng phòng ngủ. Sử dụng dữ liệu huấn luyện!

In [ ]:
# Ước tính slope và intercept để dự đoán 'price' dựa theo 'bedrooms'. Sử dụng Ctrl+C và Ctrl+V
train_beadroom = train_data['bedrooms'].values
train_price = train_data['price'].values
(intercept_bed,slope_bed) = simple_linear_regression(train_beadroom, train_price)
print("intercept and slope of bed is {:.2f} {:.2f}".format(intercept_bed,slope_bed))


intercept and slope of bed is 126751.85 123535.46


# Kiểm tra thuật toán Hồi quy Tuyến tinh

Chúng ta đã có 2 mô hình dự đoán giá nhà. Làm cách nào để biết mô hình nào tốt hơn? Tính RSS trên dữ liệu KIỂM TRA (dữ liệu này không liên quan đến việc học mô hình). Tính RSS từ dự đoán giá khi sử dụng bedrooms và squarefeet.

**Quiz: Mô hình nào (squarefeet hay bedrooms) có RSS thấp nhất trong dữ liệu KIỂM TRA? Lý do?**

In [ ]:
# Tính RSS sử dụng bedrooms trong dữ liệu KIỂM TRA. Mức độ hỗ trợ: cao.
bdrm_test_features = test_data['bedrooms'].values
bdrm_test_labels = test_data['price'].values
rss_prices_on_bedroom = get_residual_sum_of_squares(bdrm_test_features, bdrm_test_labels,intercept_bed, slope_bed)
print("Bedroom regressions result: %.2E" % rss_prices_on_bedroom)

[2 4 2 2 4]
[373822.76452777 620893.6817637  373822.76452777 373822.76452777
 620893.6817637 ]
Bedroom regressions result: 4.73E+14


In [ ]:
# Tính RSS sử dụng squarefeet trong dữ liệu KIỂM TRA. Mức độ hỗ trợ: không cao
# You have made your bed, time to lie in it
sqrt_test = test_data['sqft_living'].values
price_test = test_data['price'].values
#sqft_intercept, sqft_slope = simple_linear_regression(sqft_features, sqft_labels)
rss_prices_on_sqft = get_residual_sum_of_squares(sqrt_test, price_test,sqft_intercept, sqft_slope)
print('The RSS of predicting Prices based on Square Feet is : %.2E' % (rss_prices_on_sqft))

[1430 4670 1440 1130 3180]
[ 357817.97314081 1277876.09832343  360657.65871236  272627.40599427
  854762.94816228]
The RSS of predicting Prices based on Square Feet is : 2.68E+14


In [ ]:
print(bdrm_test_labels[0:5])
print(price_test[0:5])

[ 297000. 1578000.  562100.  631500.  780000.]
[ 297000. 1578000.  562100.  631500.  780000.]
